# Task 2: So sánh số lượng luật và độ mạnh giữa các giai đoạn

Notebook này so sánh các luật kết hợp giữa các giai đoạn thời gian khác nhau (tháng/quý) để xác định sự thay đổi về số lượng và độ mạnh của các luật.

## Mục tiêu

- Phân tích luật kết hợp theo từng tháng/quý
- So sánh số lượng luật giữa các giai đoạn
- So sánh độ mạnh (lift, confidence, support) của luật giữa các giai đoạn
- Xác định giai đoạn có nhiều luật mạnh nhất
- Phân tích sự thay đổi của các chỉ số theo thời gian


## Parameters


In [ ]:
# PARAMETERS (for papermill)

# Đường dẫn dữ liệu đã làm sạch
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"

# Đơn vị thời gian phân tích: 'month' hoặc 'quarter'
TIME_PERIOD = "month"  # hoặc "quarter"

# Tham số cho Apriori
MIN_SUPPORT = 0.01
MAX_LEN = 3
MIN_CONFIDENCE = 0.3
MIN_LIFT = 1.2

# Bật/tắt các biểu đồ
PLOT_RULES_COUNT = True
PLOT_METRICS_COMPARISON = True
PLOT_DETAILED_COMPARISON = True
PLOT_TOP_RULES_BY_PERIOD = True


## Set up


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from apriori_library import DataVisualizer

# Thiết lập style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

visualizer = DataVisualizer()


## Tải dữ liệu


In [ ]:
# Đọc dữ liệu đã làm sạch
df_clean = pd.read_csv(CLEANED_DATA_PATH, parse_dates=["InvoiceDate"])

print("=== Thông tin dữ liệu ===")
print(f"- Số giao dịch: {df_clean.shape[0]:,}")
print(f"- Khoảng thời gian: {df_clean['InvoiceDate'].min()} đến {df_clean['InvoiceDate'].max()}")
print(f"- Số hoá đơn: {df_clean['InvoiceNo'].nunique():,}")
print(f"- Số sản phẩm: {df_clean['Description'].nunique():,}")

df_clean.head()


## Phân tích luật kết hợp theo thời gian


In [ ]:
print(f"Đang phân tích luật kết hợp theo {TIME_PERIOD}...")
print("Quá trình này có thể mất vài phút...")

temporal_rules = visualizer.analyze_temporal_rules(
    df=df_clean,
    invoice_col="InvoiceNo",
    item_col="Description",
    date_col="InvoiceDate",
    time_period=TIME_PERIOD,
    min_support=MIN_SUPPORT,
    max_len=MAX_LEN,
    min_confidence=MIN_CONFIDENCE,
    min_lift=MIN_LIFT,
)

print(f"\n=== Kết quả phân tích ===")
print(f"- Số giai đoạn đã phân tích: {len(temporal_rules)}")
print(f"- Các giai đoạn: {sorted(temporal_rules.keys())}")

for period in sorted(temporal_rules.keys()):
    print(f"  - {period}: {len(temporal_rules[period])} luật")


## So sánh số lượng luật và độ mạnh giữa các giai đoạn


In [ ]:
if PLOT_RULES_COUNT:
    summary_df = visualizer.compare_periods_summary(temporal_rules)
    
    print("=== Bảng tóm tắt so sánh các giai đoạn ===")
    display(summary_df)
    
    # Vẽ biểu đồ so sánh
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Số luật
    axes[0, 0].bar(summary_df["Period"], summary_df["Số luật"], color="steelblue", edgecolor="black")
    axes[0, 0].set_title("Số lượng luật theo giai đoạn")
    axes[0, 0].set_xlabel("Giai đoạn")
    axes[0, 0].set_ylabel("Số luật")
    axes[0, 0].tick_params(axis="x", rotation=45)
    axes[0, 0].grid(True, alpha=0.3, axis="y")
    
    # Lift trung bình
    axes[0, 1].plot(summary_df["Period"], summary_df["Lift trung bình"], 
                    marker="o", linewidth=2, markersize=8, color="green")
    axes[0, 1].set_title("Lift trung bình theo giai đoạn")
    axes[0, 1].set_xlabel("Giai đoạn")
    axes[0, 1].set_ylabel("Lift trung bình")
    axes[0, 1].tick_params(axis="x", rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # Confidence trung bình
    axes[1, 0].plot(summary_df["Period"], summary_df["Confidence trung bình"], 
                    marker="s", linewidth=2, markersize=8, color="orange")
    axes[1, 0].set_title("Confidence trung bình theo giai đoạn")
    axes[1, 0].set_xlabel("Giai đoạn")
    axes[1, 0].set_ylabel("Confidence trung bình")
    axes[1, 0].tick_params(axis="x", rotation=45)
    axes[1, 0].grid(True, alpha=0.3)
    
    # Support trung bình
    axes[1, 1].plot(summary_df["Period"], summary_df["Support trung bình"], 
                    marker="^", linewidth=2, markersize=8, color="red")
    axes[1, 1].set_title("Support trung bình theo giai đoạn")
    axes[1, 1].set_xlabel("Giai đoạn")
    axes[1, 1].set_ylabel("Support trung bình")
    axes[1, 1].tick_params(axis="x", rotation=45)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


In [ ]:
if PLOT_METRICS_COMPARISON:
    summary_df = visualizer.compare_periods_summary(temporal_rules)
    
    # Vẽ biểu đồ so sánh các chỉ số trên cùng một biểu đồ
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Chuẩn hóa các giá trị để so sánh trên cùng một scale
    normalized_lift = (summary_df["Lift trung bình"] - summary_df["Lift trung bình"].min()) / (summary_df["Lift trung bình"].max() - summary_df["Lift trung bình"].min())
    normalized_conf = (summary_df["Confidence trung bình"] - summary_df["Confidence trung bình"].min()) / (summary_df["Confidence trung bình"].max() - summary_df["Confidence trung bình"].min())
    normalized_supp = (summary_df["Support trung bình"] - summary_df["Support trung bình"].min()) / (summary_df["Support trung bình"].max() - summary_df["Support trung bình"].min())
    normalized_count = (summary_df["Số luật"] - summary_df["Số luật"].min()) / (summary_df["Số luật"].max() - summary_df["Số luật"].min())
    
    ax.plot(summary_df["Period"], normalized_lift, marker="o", linewidth=2, markersize=8, label="Lift (chuẩn hóa)", color="green")
    ax.plot(summary_df["Period"], normalized_conf, marker="s", linewidth=2, markersize=8, label="Confidence (chuẩn hóa)", color="orange")
    ax.plot(summary_df["Period"], normalized_supp, marker="^", linewidth=2, markersize=8, label="Support (chuẩn hóa)", color="red")
    ax.plot(summary_df["Period"], normalized_count, marker="d", linewidth=2, markersize=8, label="Số luật (chuẩn hóa)", color="blue")
    
    ax.set_title("So sánh các chỉ số (đã chuẩn hóa) theo giai đoạn")
    ax.set_xlabel("Giai đoạn")
    ax.set_ylabel("Giá trị chuẩn hóa (0-1)")
    ax.tick_params(axis="x", rotation=45)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Bảng so sánh chi tiết
    print("\n=== Bảng so sánh chi tiết ===")
    print(f"Giai đoạn có nhiều luật nhất: {summary_df.loc[summary_df['Số luật'].idxmax(), 'Period']} ({summary_df['Số luật'].max()} luật)")
    print(f"Giai đoạn có ít luật nhất: {summary_df.loc[summary_df['Số luật'].idxmin(), 'Period']} ({summary_df['Số luật'].min()} luật)")
    print(f"\nGiai đoạn có Lift trung bình cao nhất: {summary_df.loc[summary_df['Lift trung bình'].idxmax(), 'Period']} ({summary_df['Lift trung bình'].max():.2f})")
    print(f"Giai đoạn có Lift trung bình thấp nhất: {summary_df.loc[summary_df['Lift trung bình'].idxmin(), 'Period']} ({summary_df['Lift trung bình'].min():.2f})")
    print(f"\nGiai đoạn có Confidence trung bình cao nhất: {summary_df.loc[summary_df['Confidence trung bình'].idxmax(), 'Period']} ({summary_df['Confidence trung bình'].max():.2f})")
    print(f"Giai đoạn có Confidence trung bình thấp nhất: {summary_df.loc[summary_df['Confidence trung bình'].idxmin(), 'Period']} ({summary_df['Confidence trung bình'].min():.2f})")


## Phân tích phân phối các chỉ số


In [ ]:
if PLOT_DETAILED_COMPARISON:
    # Thu thập tất cả các giá trị lift, confidence, support từ tất cả các giai đoạn
    all_lifts = []
    all_confidences = []
    all_supports = []
    all_periods = []
    
    for period, rules_df in temporal_rules.items():
        all_lifts.extend(rules_df["lift"].tolist())
        all_confidences.extend(rules_df["confidence"].tolist())
        all_supports.extend(rules_df["support"].tolist())
        all_periods.extend([period] * len(rules_df))
    
    comparison_df = pd.DataFrame({
        "Period": all_periods,
        "Lift": all_lifts,
        "Confidence": all_confidences,
        "Support": all_supports
    })
    
    # Vẽ boxplot so sánh
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Boxplot Lift
    comparison_df.boxplot(column="Lift", by="Period", ax=axes[0])
    axes[0].set_title("Phân phối Lift theo giai đoạn")
    axes[0].set_xlabel("Giai đoạn")
    axes[0].set_ylabel("Lift")
    axes[0].tick_params(axis="x", rotation=45)
    
    # Boxplot Confidence
    comparison_df.boxplot(column="Confidence", by="Period", ax=axes[1])
    axes[1].set_title("Phân phối Confidence theo giai đoạn")
    axes[1].set_xlabel("Giai đoạn")
    axes[1].set_ylabel("Confidence")
    axes[1].tick_params(axis="x", rotation=45)
    
    # Boxplot Support
    comparison_df.boxplot(column="Support", by="Period", ax=axes[2])
    axes[2].set_title("Phân phối Support theo giai đoạn")
    axes[2].set_xlabel("Giai đoạn")
    axes[2].set_ylabel("Support")
    axes[2].tick_params(axis="x", rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Thống kê mô tả
    print("\n=== Thống kê mô tả theo giai đoạn ===")
    stats_by_period = comparison_df.groupby("Period").agg({
        "Lift": ["mean", "std", "min", "max"],
        "Confidence": ["mean", "std", "min", "max"],
        "Support": ["mean", "std", "min", "max"]
    })
    display(stats_by_period.round(4))


## Top luật theo từng giai đoạn


In [ ]:
if PLOT_TOP_RULES_BY_PERIOD:
    top_n = 5
    
    print(f"=== Top {top_n} luật theo Lift trong mỗi giai đoạn ===\n")
    
    for period in sorted(temporal_rules.keys()):
        rules_df = temporal_rules[period]
        top_rules = rules_df.nlargest(top_n, "lift")[["rule_str", "lift", "confidence", "support"]]
        
        print(f"\n{period}:")
        print("-" * 80)
        for idx, row in top_rules.iterrows():
            print(f"  {row['rule_str']}")
            print(f"    Lift: {row['lift']:.2f}, Confidence: {row['confidence']:.2f}, Support: {row['support']:.4f}")
    
    # Vẽ biểu đồ so sánh top luật
    fig, axes = plt.subplots(len(temporal_rules), 1, figsize=(14, 4 * len(temporal_rules)))
    
    if len(temporal_rules) == 1:
        axes = [axes]
    
    for idx, period in enumerate(sorted(temporal_rules.keys())):
        rules_df = temporal_rules[period]
        top_rules = rules_df.nlargest(top_n, "lift")
        
        axes[idx].barh(range(len(top_rules)), top_rules["lift"].values, color="steelblue")
        axes[idx].set_yticks(range(len(top_rules)))
        axes[idx].set_yticklabels([rule[:60] + "..." if len(rule) > 60 else rule 
                                   for rule in top_rules["rule_str"].values], fontsize=9)
        axes[idx].set_title(f"Top {top_n} luật theo Lift - {period}")
        axes[idx].set_xlabel("Lift")
        axes[idx].invert_yaxis()
        axes[idx].grid(True, alpha=0.3, axis="x")
    
    plt.tight_layout()
    plt.show()


## Kết luận về so sánh luật giữa các giai đoạn

Dựa trên phân tích trên, chúng ta có thể rút ra các nhận định sau:

### 1. Số lượng luật

- **Giai đoạn có nhiều luật nhất**: Xác định được giai đoạn nào có nhiều luật kết hợp nhất
- **Giai đoạn có ít luật nhất**: Xác định được giai đoạn nào có ít luật nhất
- **Xu hướng**: Phân tích xem số lượng luật có tăng/giảm theo thời gian không

### 2. Độ mạnh của luật (Lift)

- **Giai đoạn có Lift cao nhất**: Xác định giai đoạn có luật mạnh nhất
- **Biến thiên**: Phân tích sự thay đổi của Lift giữa các giai đoạn
- **Pattern**: Xác định pattern theo mùa (nếu có)

### 3. Confidence và Support

- **Confidence**: Độ tin cậy của luật thay đổi như thế nào giữa các giai đoạn
- **Support**: Tần suất xuất hiện của luật có thay đổi theo thời gian không

### 4. Ứng dụng thực tế

- **Chiến lược cross-selling**: Tập trung vào các luật mạnh trong từng giai đoạn
- **Quản lý hàng tồn kho**: Dựa trên số lượng và độ mạnh luật để dự trữ hàng
- **Marketing**: Thiết kế chiến dịch dựa trên các luật mạnh theo từng giai đoạn
